In [8]:
# import module
from datetime import datetime
from datetime import timedelta
import requests
import os
import json
import Bearer
import pytz
import time

import pandas as pd

In [9]:
# 時刻を変換する関数

def make_local_dt (dt):
    utc_dt = dt - timedelta(hours=+9)
    return (pytz.utc.localize(utc_dt).astimezone(pytz.timezone("Asia/Tokyo")) )

    
def dt2localiso(dt):
    # この段階で
    if dt is None:
        return ''
    
    return dt.isoformat()

def iso_to_localdt(iso_str):
    dt = None
    try:
        dt = datetime.strptime(iso_str, '%Y-%m-%dT%H:%M:%S.%fZ')
        dt = pytz.utc.localize(dt).astimezone(pytz.timezone("Asia/Tokyo"))
    except ValueError:
        try:
            dt = datetime.strptime(iso_str, '%Y-%m-%dT%H:%M:%S.%f%z')
            dt = dt.astimezone(pytz.timezone("Asia/Tokyo"))
        except ValueError:
            pass
    return dt

In [10]:
start_time_dt = make_local_dt(datetime(2020, 10, 16, 0, 0, 0) )
movie_name = str(start_time_dt.date())+"_"+"鬼滅の刃"

# 終了日
end_time_dt = start_time_dt + timedelta(days=88) # 2020年10月16日スタート

start_time = dt2localiso(start_time_dt)
end_time = dt2localiso(end_time_dt)

In [11]:
import requests
import os
import json
import Bearer

# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'

def auth():
    return Bearer.BEARER
#     return os.environ.get("BEARER_TOKEN")


def create_url(users):
    # Specify the usernames that you want to lookup below
    # You can enter up to 100 comma-separated values.
#     usernames = "usernames=TwitterDev,TwitterAPI"
    user_fields = "user.fields=description,created_at"
    # User fields are adjustable, options include:
    # created_at, description, entities, id, location, name,
    # pinned_tweet_id, profile_image_url, protected,
    # public_metrics, url, username, verified, and withheld
    url = "https://api.twitter.com/2/users?ids="+users+"&"+user_fields
    return url

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def connect_to_endpoint(url, headers):
    response = requests.request("GET", url, headers=headers)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()


In [12]:
user_ids_list = [4160473333, 909365712904380416]
users_ids_txt = ','.join(map(str,user_ids_list))

bearer_token = auth()
url = create_url(users_ids_txt)
headers = create_headers(bearer_token)
json_response = connect_to_endpoint(url, headers)

print(json.dumps(json_response, indent=4, sort_keys=True))

200
{
    "data": [
        {
            "created_at": "2015-11-07T19:48:34.000Z",
            "description": "\u4e2d\u9014\u534a\u7aef\u306a\u30f2\u30bf\u3002\u57fa\u672c\u30ce\u30de\u30ab\u30d7\u63a8\u3057\u3001\u305f\u307e\u306b\u6c17\u304c\u5411\u3044\u305f\u3068\u304d\u3060\u3051\u8150\u3084\u767e\u5408\u3082\u3064\u307e\u307f\u98df\u3044\u3057\u3066\u308b\u3002\n\u300e\u30d7\u30ea\u30c1\u30e3\u30f3\u300f\u300e\u30a2\u30a4\u30ab\u30c4!\u300f\u7b49\u306e\u5973\u5150\u5411\u3051\u30b2\u30fc\u30e0\u7b50\u4f53\u3067\u904a\u3076\u3053\u3068\u3082\u3002\u3000\n\u6700\u8fd1\u30cf\u30de\u3063\u3066\u308b\u306e\u306f\u3001\u300eKING OF PRISM\u300f(\u30d2\u30ed\u69d8\u6700\u63a8\u3057\u30fb\u30b7\u30f3\u3061\u3083\u3093\u63a8\u3057)\u3068\u300e\u30d3\u30c3\u30ab\u30e1\u5a18\u300f(\u65b0\u5bbf\u7d44&\u540d\u53e4\u5c4b\u7d44\u63a8\u3057)\u3002",
            "id": "4160473333",
            "name": "\u307e\u308d\u306b\u3043",
            "username": "maronny2rain"
        },
        {
        

In [13]:
df = pd.read_csv("./data/kimetsu_tweet_merged.csv",header=None, names=["tweet_id","date","text","like","retweet","author_id"] )
author_ids = list(set(df.author_id))


import re
author_ids = [i for i in author_ids if type(i) == str and  re.match(r'^[0-9]{1,19}$', i )]
# for id_ in author_ids:
#     if not re.match(r'^[0-9]{1,19}$', id_ ):
#         print(id_)

In [14]:
dict_for_df = {}
import time

def get_user_data(user_id_list):
    users_ids_txt = ','.join(map(str,user_id_list)) 
    bearer_token = auth()
    url = create_url(users_ids_txt)
    headers = create_headers(bearer_token)
    json_response = connect_to_endpoint(url, headers)
    return (json_response)

for i, batch_idx in enumerate(range(100, len(author_ids), 100)):
    print (batch_idx)
    before_idx = batch_idx-100
    tmp_author_ids = author_ids[before_idx: batch_idx]

    json_response = get_user_data(tmp_author_ids)

    time.sleep(10)
    
    for j, tmp_user in enumerate( json_response["data"]):
        tmp_se = pd.Series( [ tmp_user["created_at"], tmp_user["description"], tmp_user["id"], tmp_user["name"], tmp_user["username"] ], 
                           index=['created_at','description', 'id','name','username'] )

        dict_for_df[j+before_idx]=tmp_se

    


100
200
200
200
300
200
400
200
500
200
600
200
700
200
800
200
900
200
1000
200
1100
200
1200
200
1300
200
1400
200
1500
200
1600
200
1700
200
1800
200
1900
200
2000
200
2100
200
2200
200
2300
200
2400
200
2500
200
2600
200
2700
200
2800
200
2900
200
3000
200
3100
200
3200
200
3300
200
3400
200
3500
200
3600
200
3700
200
3800
200
3900
200
4000
200
4100
200
4200
200
4300
200
4400
200
4500
200
4600
200
4700
200
4800
200
4900
200
5000
200
5100
200
5200
200
5300
200
5400
200
5500
200
5600
200
5700
200
5800
200
5900
200
6000
200
6100
200
6200
200
6300
200
6400
200
6500
200
6600
200
6700
200
6800
200
6900
200
7000
200
7100
200
7200
200
7300
200
7400
200
7500
200
7600
200
7700
200
7800
200
7900
200
8000
200
8100
200
8200
200
8300
200
8400
200
8500
200
8600
200
8700
200
8800
200
8900
200
9000
200
9100
200
9200
200
9300
200
9400
200
9500
200
9600
200
9700
200
9800
200
9900
200
10000
200
10100
200
10200
200
10300
200
10400
200
10500
200
10600
200
10700
200
10800
200
10900
200
11000
200
11100
20

200
87400
200
87500
200
87600
200
87700
200
87800
200
87900
200
88000
200
88100
200
88200
200
88300
200
88400
200
88500
200
88600
200
88700
200
88800
200
88900
200
89000
200
89100
200
89200
200
89300
200
89400
200
89500
200
89600
200
89700
200
89800
200
89900
200
90000
200
90100
200
90200
200
90300
200
90400
200
90500
200
90600
200
90700
200
90800
200
90900
200
91000
200
91100
200
91200
200
91300
200
91400
200
91500
200
91600
200
91700
200
91800
200
91900
200
92000
200
92100
200
92200
200
92300
200
92400
200
92500
200
92600
200
92700
200
92800
200
92900
200
93000
200
93100
200
93200
200
93300
200
93400
200
93500
200
93600
200
93700
200
93800
200
93900
200
94000
200
94100
200


In [15]:
df_prof = pd.DataFrame.from_dict(dict_for_df, orient="index")

In [16]:
df_prof

,created_at,description,id,name,username
0,2011-01-08T11:39:54.000Z,フリーフォトグラファー /アー写や宣材撮影、ライブ、舞台ビジュアル撮影たまにブライダルなど。...,235517449,NORI,NORINGO26
1,2018-10-25T14:27:34.000Z,,1055465897219878913,merrick10535,fOvtkVqK2Lfw12r
2,2015-01-31T12:07:00.000Z,サンキュー遠北\nフォーエバー遠北\n\n金クレア＾～：https://t.co/M7PqX...,3004809181,こをたく@大剣豪⚔🗡,kw_tk1
3,2018-02-06T06:02:35.000Z,懸賞大好き主婦です,960755580775276544,HAKAちゃん,HAKA86324253
4,2013-04-04T03:48:04.000Z,日常な呟きやアニメやゲームの呟きをしてます|'ㅂ')テイルズ(推し:ｴﾄﾞﾅ/ｱｲｾﾞﾝ/ﾕ...,1326081427,Nuco๑⃙⃘˙∞˙๑⃙⃘),nuco_184
...,...,...,...,...,...
94093,2017-10-18T11:46:48.000Z,声優・アニメ・映画が大好き❗┃銀魂┃鬼滅の刃┃呪術廻戦┃キングダム┃名探偵コナン┃シティーハ...,920617155275636736,malekith,MCU_Marvel_3000
94094,2015-03-14T05:13:08.000Z,,3079446818,ガミ,Gami5892
94095,2019-06-01T13:21:41.000Z,TUBE大好きmrtラジオ📻のリスナーです。JOY FMは耳恋 ・WEEKEND JAM・Ｏ...,1134812261044768769,なるとさいふ,narutosaihu7610
94096,2018-04-11T16:23:57.000Z,褐色が好きなだけの人 アイコン（@kyoro_ina）様 ヘッダー（@uminekomr）様...,984104777658191873,こおり,koori_qooi


In [17]:
df_prof.to_excel("result0802.xlsx")

In [18]:
# df

In [19]:
# ids = [2987256462,1073200913056071680,744881541457682432,1005772426393735169,1312722977755021312,1296292806588116992,331441514,1061934749566922752,1300279569987956737,1243804231334629377,2699769367,3229767464,32331287,1124842093057167360,1464470156,1251678204684677121,949327761696423936,4911443174,1311529669464735744,798349045932507136,2313524420,848871591232454658,721441813,185006673,123572236,1181113149073739778,864535750544470016,1326949625060761601,609122548,973580980677001216,1308592258145382400,995056578695909376,95769450,3724459818,2928485581,1280139281541894150,1267716136100978689,302325090,892008605175631875,762999381796593664,1204751156171292673,709848680,2574686574,2661576594,96958487,1166299695288729600,1626543992,777933586578051074,975446780350316544,1214359771739869184,110981657,1318143672802713601,2558496764,3313117790,819834679574417409,1289194048289267713,1241714137278881793,878250774274363393,1302205773997510656,2391320443,133572334,1107268822757629952,45458166,1314746393613881344,314909533,1964841312,1278631904061734912,1217142018511474689,2373640014,272836618,3269843726,825951069834391552,2823976416,3003624031,1240334144,1092047192951422983,2840976566,995286741517938688,2869021716,1004225998844379136,835347230999506944,856836785246269442,1153676831045771264,997120626337828865,3171080930,2281942154,1217793463883722752,953104264569606144,129250622,1123347944919392257,1677456487,1290559288134340610,1002078980474716160,1298897840,1270746886396239873,3302866260,707836924552024064,1148607253131325440,176772671,2325645582]

In [20]:
# get_user_data(ids)